In [35]:
import json
import pandas as pd
from pyspark import SparkContext
from pyspark.ml import Pipeline
from pyspark.ml.feature import HashingTF, Tokenizer, StopWordsRemover, ChiSqSelector, IDF
from pyspark.sql import SparkSession


In [36]:
spark = SparkSession.builder.config("spark.driver.memory", "4g").getOrCreate()

In [38]:
file = open("./reviews_devset copy.json")

reviews_devset = file.readlines()


reviewsDF = pd.DataFrame.from_records(list(map(json.loads,reviews_devset)))

print(reviewsDF)

reviewsDF["category"] = pd.Categorical(reviewsDF["category"])
reviewsDF["labels"] = reviewsDF["category"].cat.codes
print(reviewsDF.loc[:,["reviewText","category","labels"]])

       reviewerID        asin      reviewerName helpful  \
0  A1OX7LOQV61DWM  B00FT8ZHHE  Rachel L. Hrdina  [0, 1]   
1  A1CF0N2YFQVRLW  B00FV58KB0               Liz  [0, 0]   
2  A2KCSL0QE7JMA1  B00FWWWVKS   Marlaine Chenal  [0, 1]   

                                          reviewText  overall  \
0  I love the design. I love the large digitial n...      5.0   
1  What an exciting concept & product!  Excellent...      4.0   
2  excellent sticking surface, pliable and light....      4.0   

                       summary  unixReviewTime   reviewTime  \
0             Love this scale!      1388534400   01 1, 2014   
1  100% Pure Garcinia Cambogia      1403827200  06 27, 2014   
2                very pleased.      1392422400  02 15, 2014   

                   category  
0  Health_and_Personal_Care  
1  Health_and_Personal_Care  
2  Health_and_Personal_Care  
                                          reviewText  \
0  I love the design. I love the large digitial n...   
1  What an exciti

In [40]:
spark = SparkSession.builder.master("local[1]").appName("part2ex2").getOrCreate()

spark_dff = spark.createDataFrame(reviewsDF.loc[:,["reviewText","category","labels"]])

In [43]:
tokenizer = Tokenizer(inputCol="reviewText", outputCol="rawWords")

remover = StopWordsRemover(
    inputCol=tokenizer.getOutputCol(),
    outputCol="words",
    caseSensitive=False)

hashingTF = HashingTF(
    inputCol=remover.getOutputCol(), outputCol="rawFeatures", numFeatures=2000)

idf = IDF(inputCol=hashingTF.getOutputCol(), outputCol="features")

selector = ChiSqSelector(
    numTopFeatures=2000, 
    featuresCol=idf.getOutputCol(),
    outputCol="selectedFeatures", 
    labelCol="labels"
    )

pipeline = Pipeline(stages=[tokenizer, remover, hashingTF, idf, selector])

pipeline = Pipeline(stages=[tokenizer])

model = pipeline.fit(spark_dff)



In [45]:
out = model.transform(spark_dff)

In [49]:
out.show()

Py4JJavaError: An error occurred while calling o737.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 7.0 failed 1 times, most recent failure: Lost task 0.0 in stage 7.0 (TID 7) (128.130.249.198 executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:192)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:166)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:364)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:328)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoi

In [50]:
out.toPandas().to_csv("test.csv")

Py4JJavaError: An error occurred while calling o737.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 8.0 failed 1 times, most recent failure: Lost task 0.0 in stage 8.0 (TID 8) (128.130.249.198 executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:192)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:166)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:364)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:328)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadChe